In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,layers,optimizers,Sequential,metrics

In [4]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [5]:
tf.__version__

'2.1.0'

In [6]:
class ResBlock(layers.Layer):
  def __init__(self, filter_nums, strides=1, residual_path=False):
      super(ResBlock, self).__init__()

      self.conv_1 = layers.Conv2D(filter_nums,(3,3),strides=strides,padding='same')
      self.bn_1 = layers.BatchNormalization()
      self.act_relu = layers.Activation('relu')

      self.conv_2 = layers.Conv2D(filter_nums,(3,3),strides=1,padding='same')
      self.bn_2 = layers.BatchNormalization()
      
      if strides !=1:
        self.block = Sequential()
        self.block.add(layers.Conv2D(filter_nums,(1,1),strides=strides))
      else:
        self.block = lambda x:x


  def call(self, inputs, training=None):

      x = self.conv_1(inputs)
      x = self.bn_1(x, training=training)
      x = self.act_relu(x)
      x = self.conv_2(x)
      x = self.bn_2(x,training=training)
      
      identity = self.block(inputs)
      outputs = layers.add([x,identity])
      # outputs = layers.add([x])
      # outputs = x
      outputs = tf.nn.relu(outputs)

      return outputs

In [7]:
class ResNet(keras.Model):
  def __init__(self,layers_dims,nums_class=10):
    super(ResNet,self).__init__()

    self.model = Sequential([layers.Conv2D(64,(3,3),strides=(1,1)),
                             layers.BatchNormalization(),
                             layers.Activation('relu'),
                             layers.MaxPooling2D(pool_size=(2,2),strides=(1,1),padding='same')])

    self.layer_1 = self.ResNet_build(64,layers_dims[0])
    self.layer_2 = self.ResNet_build(128,layers_dims[1],strides=2)   
    self.layer_3 = self.ResNet_build(256,layers_dims[2],strides=2) 
    self.layer_4 = self.ResNet_build(512,layers_dims[3],strides=2)   

    self.avg_pool = layers.GlobalAveragePooling2D()                 
    self.fc_model = layers.Dense(nums_class)    

  def call(self, inputs, training=None):
    x = self.model(inputs)
    x = self.layer_1(x)
    x = self.layer_2(x)  
    x = self.layer_3(x) 
    x = self.layer_4(x)  
    x = self.avg_pool(x) 
    x = self.fc_model(x)
    return x

  def ResNet_build(self,filter_nums,block_nums,strides=1):
    build_model = Sequential()
    build_model.add(ResBlock(filter_nums,strides))
    for _ in range(1,block_nums):
      build_model.add(ResBlock(filter_nums,strides=1))
    return build_model

In [8]:
def ResNet18():
  return ResNet([2,2,2,2])

In [9]:
(x,y),(x_test,y_test) = datasets.cifar10.load_data()
y = tf.squeeze(y,axis=1)
y_test = tf.squeeze(y_test,axis=1)
print(x.shape,y.shape)

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3) (50000,)


In [10]:
(x,y),(x_test,y_test) = datasets.cifar10.load_data()
# Squeeze前的資料維度
print(y.shape,y_test.shape)
# Squeeze後的資料維度
y = tf.squeeze(y,axis=1)
y_test = tf.squeeze(y_test,axis=1)
print(y.shape,y_test.shape)

# 最終轉換後的資料維度
print(x.shape,y.shape)

(50000, 1) (10000, 1)
(50000,) (10000,)
(50000, 32, 32, 3) (50000,)


In [11]:
def feature_scale(x,y):
  x = tf.cast(x,dtype=tf.float32)/255.
  y = tf.cast(y,dtype=tf.int32)
  return x,y

In [12]:
ResNet_model = ResNet18()
ResNet_model.build(input_shape=(None,32,32,3))
optimizer = optimizers.Adam(lr=1e-3)

In [13]:
ResNet_model.summary()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  2048      
_________________________________________________________________
sequential_1 (Sequential)    multiple                  148736    
_________________________________________________________________
sequential_2 (Sequential)    multiple                  526976    
_________________________________________________________________
sequential_4 (Sequential)    multiple                  2102528   
_________________________________________________________________
sequential_6 (Sequential)    multiple                  8399360   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  5130

In [14]:
data = tf.data.Dataset.from_tensor_slices((x,y))
data = data.map(feature_scale).shuffle(10000).batch(512)

data_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
data_test = data_test.map(feature_scale).batch(512) 

In [15]:
data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

(512, 32, 32, 3) (512,)


In [16]:
for i in range(10):
  for step,(x,y) in enumerate(data):
    with tf.GradientTape() as tape:
      logits = ResNet_model(x)
      y_one_hot = tf.one_hot(y,depth=10)
      loss = tf.losses.categorical_crossentropy(y_one_hot,logits,from_logits=True)
      loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss,ResNet_model.trainable_variables)
    optimizer.apply_gradients(zip(grads,ResNet_model.trainable_variables))
    
    if step %100==0:
      print(i,step,'loss:',float(loss))
  total_loss = 0
  total_num=0
  for x,y in data_test:

    logits = ResNet_model(x)
    
    prob = tf.nn.softmax(logits,axis=1)
    pred = tf.argmax(prob,axis=1)

    pred = tf.cast(pred,dtype=tf.int32)
    correct = tf.equal(pred,y)

    result = tf.reduce_sum(tf.cast(correct,dtype=tf.int32))

    total_loss += int(result)
    total_num += x.shape[0]

  acc = total_loss/total_num
  print(i,'acc:',acc)

0 0 loss: 2.3052592277526855
0 acc: 0.3157
1 0 loss: 1.8333091735839844
1 acc: 0.4506
2 0 loss: 1.4909344911575317
2 acc: 0.544
3 0 loss: 1.2291452884674072
3 acc: 0.578
4 0 loss: 1.1379151344299316
4 acc: 0.65
5 0 loss: 0.8637693524360657
5 acc: 0.6701
6 0 loss: 0.6892322301864624
6 acc: 0.6776
7 0 loss: 0.5811546444892883
7 acc: 0.6863
8 0 loss: 0.3595947027206421
8 acc: 0.6864
9 0 loss: 0.24096521735191345
9 acc: 0.6701


In [17]:
total_loss = 0
total_num=0
for x,y in data_test:

  gd = ResNet_model(x)
  prob = tf.nn.softmax(gd,axis=1)
  pred = tf.argmax(prob,axis=1)
  
  pred = tf.cast(pred,dtype=tf.int32)
  correct = tf.equal(pred,y)
  
  result = tf.reduce_sum(tf.cast(correct,dtype=tf.int32))
  
  total_loss += int(result)
  total_num += x.shape[0]
  
acc = total_loss/total_num
print(acc)

0.6701
